In [9]:
# -*- coding: utf-8 -*-
"""
This script is used to reverse geocode health sources which do not have street addresses
but do have lat/lon coordinates. It uses the OSM Nominatim API.

It filters out results results that have already been geocoded so that new data 
can be added to the pipeline without re-processing each record.

This script is adapted from Education Facilities to Healthcare.
"""

import requests
import json
import pandas as pd
import numpy as np
import time
from os.path import exists
from datetime import datetime

# temporarily suppresses SettingWithCopyWarning
pd.options.mode.chained_assignment = None 

In [10]:
# load new input data and (optionally) output data from previous runs 

input_data = "geocoded_2022-04-21.csv"
prev_reverse_geocoded_data = ""

df_input = pd.read_csv(input_data, low_memory=False, dtype="str")

# detect previously geocoded rows from previous output
if exists(prev_reverse_geocoded_data):
    df_previous_run = pd.read_csv(prev_reverse_geocoded_data, low_memory=False, dtype="str")
    remove_list = ['osm_reverse', 'no_osm', 'osm_facility_name', 'osm_address', 'gc_street_address', 'osm_city']
    df_previously_r_geocoded = df_previous_run[df_previous_run["geo_source"].isin(remove_list)]    
    print(str(len(df_previously_r_geocoded)) + ' records already reverse geocoded in previous run')
else:
    df_previously_r_geocoded = None
    print('no previous csv detected')

no previous csv detected


In [11]:
# replace empty row values with na
df_input = df_input.replace(r'^\s*$', np.nan, regex=True)

In [38]:
len(df_input)

27323

In [12]:
print("missing street addresses: " + str(df_input['street_addr'].isnull().sum()))

missing street addresses: 4207


In [13]:
# filter input data for rows that have already been reverse geocoded, based on 'temp_id'

if isinstance(df_previously_r_geocoded, pd.DataFrame):
    geo_list = list(df_previously_r_geocoded['temp_id'])
#     print("filenames already coded: " + str(set(geo_list)))
    # split df
        # df = input that has a filename than isn't in the 
        # already coded list
        
        # df_leftover1 = previously geocoded data filtered
        # to just those which are marked as coded
    
    df = df_input[~df_input['temp_id'].isin(geo_list)]
    df_previously_coded = df_previous_run[df_previous_run['temp_id'].isin(geo_list)]
#     print(str(len(df_input) - len(df)) + ' records excluded from reverse geocoding based on filename')
else:
    print('no previously geocoded results found in dataframe')
    df_previously_coded = pd.DataFrame()
    df = df_input
    
print("missing street addresses: " + str(df['street_addr'].isnull().sum()))

# filter df to those without street address but with street name and street number
df_geo_source = df[(df.street_addr.notnull())]
df_no_address = df[(df.street_addr.isnull())]

# create new street address from street address and number
# df_concat_address = those which no street_addr, but do have both street name and street number
df_concat_address = df_no_address[(df_no_address.street_name.notnull()) & (df_no_address.street_no.notnull())]
df_concat_address['street_addr'] = df_concat_address['street_addr'].fillna(df_concat_address['street_no'].astype(str) + " " + df_concat_address['street_name'])
print("number of addresses created from other columns: " + str(len(df_concat_address)))


# df_leftover = results that do not need geocoding
# = those previously coded, those with newly concatenated results, 
# and those with geo_source already given 
df_leftover = pd.concat([df_previously_coded, df_concat_address, df_geo_source])

# df = just those with no street_addr, which also have a missing value for streen_name or streen_no
df = df_no_address[(df_no_address.street_name.isnull()) | (df_no_address.street_no.isnull())]


print(str(len(df)) + " records left to reverse geocode")

# sanity check for length of dataframes
diff = len(df_input) - len(df_leftover) - len(df)
if diff != 0:
    print('ERROR there are ' + str(-diff) + ' extra records. review the scripts to correct')

no previously geocoded results found in dataframe
missing street addresses: 4207
number of addresses created from other columns: 0
4207 records left to reverse geocode


In [14]:
# define parameters for osm api call
headers = {
    'User-Agent': 'Sam Lumley, Statistics Canada',
    'From': 'sam.lumley@statcan.gc.ca' 
    }

url = 'https://nominatim.openstreetmap.org/reverse?'

JSONS = []
JSONS_CITIES = []

# define our api queries

lats = list(df['latitude'])
lons = list(df['longitude'])

In [ ]:
# make a nominatim reverse geocode request for each record 

# first batch 0-1703

# for i in range(5): # use this line for testing small batches
for i in range(len(lats) - 1704):
    
    i = i + 1704
    
    # temp, get last five. so from len(lats) - 5 to len(lats)
#     i = i + len(lats) - 5
    
    lat = lats[i]
    lon = lons[i]
    print(str(i) + ": lat: " + str(lat) + ", lon: " + str(lon))
    
    request_timing = 3 # seconds between api requests
            
    params = {'lat': lat,
            'lon': lon,
            'format':'json',
            'email':'sam.lumley@statcan.gc.ca'}
    time.sleep(request_timing) 
    coords = requests.get(url, params=params, headers=headers)
    resp = coords.json()
        
    if len(resp) > 0:
        print('osm address found')
        df['geo_source'].iloc[i] = "osm_reverse"
    else:
        print('no osm address found')
        
                
    JSONS.append(resp)

#     if resp!=[]:
#         print(resp)

In [23]:
with open('reverse_nominatim_0-4207.json', 'w', encoding='utf-8') as f:
    json.dump(JSONS, f, ensure_ascii=False, indent=4) 

In [30]:
len(JSONS)

# doing things in batches
# first batch up to 1704

4105

In [33]:
# read json request results into our dataframe

def append_blank(index):
    df['geo_source'].iloc[index] = "no_osm"
    LATS.append('')
    LONS.append('')
    NAME.append('')
    ST_NO.append('')
    ST_NAME.append('')
    CITY.append('')
    PROV.append('')
    POST.append('')
    COUNTRY.append('')
    TYPE.append('')
    CLASS.append('')

with open('reverse_nominatim_0-4207.json', 'r', encoding='utf-8') as f:
    JSONS=json.load(f)    

LATS = []
LONS = []
NAME = []
ST_NO = []
ST_NAME = []
CITY = []
PROV = []
POST = []
COUNTRY = []
TYPE = []
CLASS = []


for index, element in enumerate(JSONS):
    if element==[]:
        my_function()
    else:
        # For now we will use everything osm gives us
        # later we might want to filter by class - eg just to "amenity" 
        
#         if (element['address']['country_code']=='ca') and ('amenity' in element['address'].keys()): 
#         if (element['address']['country_code']=='ca') and ((element['class'] in ['amenity', 'place', 'building']) ):
            
        if 'address' in element:
            if 'country_code' in element['address']:
                if (element['address']['country_code']=='ca'):
                    df['geo_source'].iloc[index] = "osm_reverse"
                    COUNTRY.append(element['address']['country_code'])
                    LATS.append(element['lat'])
                    LONS.append(element['lon'])
                    
                    if 'amenity' in element.keys():
                        NAME.append(element['address']['amenity'])
                    else:
                        NAME.append('')
                        
                    if 'house_number' in element['address']:
                        ST_NO.append(element['address']['house_number'])
                    else:
                        ST_NO.append('')    
                        
                    if 'road' in element['address']:
                        ST_NAME.append(element['address']['road'])
    #                     df['geo_source'].iloc[index] = "osm_street"
                    else:
                        ST_NAME.append('') 
                
                    if 'city' in  element['address']:
                        CITY.append(element['address']['city'])
                    else:
                        CITY.append('')    

                    if 'state' in element.keys():
                        PROV.append(element['address']['state'])
                    else:
                        PROV.append('')

                    if 'postcode' in element['address']:
                        POST.append(element['address']['postcode'])
                    else:
                        POST.append('')

                    COUNTRY.append(element['address']['country_code'])

                    if 'type' in element.keys():
                        TYPE.append(element['type'])
                    else:
                        TYPE.append('')
                    if 'class' in element.keys():
                        CLASS.append(element['class'])
                    else:
                        CLASS.append('')  
                        
                else:
                    append_blank(index)
                        
            else:
                append_blank(index)
                        
        else:
            append_blank(index)

# append results to dataframe
df['osm_name'] = NAME
df['osm_street_no'] = ST_NO
df['osm_street_name'] = ST_NAME
df['osm_city'] = CITY
df['osm_province'] = PROV
df['osm_postal_code'] = POST
df['osm_class'] = CLASS
df['osm_type'] = TYPE
df['osm_lat'] = LATS
df['osm_lon'] = LONS        

In [34]:
df['geo_source'].value_counts()

osm_reverse    4115
no_osm           92
Name: geo_source, dtype: int64

In [35]:
len(JSONS)

4207

In [36]:
# recombine datasets: (1) no reverse geocode needed
# (2) previously reverse geocoded (3) newly reverse geocoded
df_everything = pd.concat([df_leftover, df])

In [37]:
len(df_everything)

27323

In [39]:
date = str(datetime.today().strftime('%Y-%m-%d'))
filename = "reverse_geocoded_" + date + ".csv"
df_everything.to_csv(filename, index=False)

In [40]:
df_everything['geo_source'].value_counts()

Source               20937
osm_reverse           4115
osm_address            986
osm_facility_name      588
osm_city               404
no_osm                 290
gc_street_address        3
Name: geo_source, dtype: int64